In [1]:
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import optuna

In [2]:
train=pd.read_csv('train_0OECtn8.csv')
test=pd.read_csv('test_1zqHu22.csv')

In [3]:
train

,row_id,user_id,category_id,video_id,age,gender,profession,followers,views,engagement_score
0,1,19990,37,128,24,Male,Student,180,1000,4.33
1,2,5304,32,132,14,Female,Student,330,714,1.79
2,3,1840,12,24,19,Male,Student,180,138,4.35
3,4,12597,23,112,19,Male,Student,220,613,3.77
4,5,13626,23,112,27,Male,Working Professional,220,613,3.13
...,...,...,...,...,...,...,...,...,...,...
89192,89193,23996,15,32,25,Male,Other,340,662,3.91
89193,89194,20466,20,47,31,Male,Other,240,892,3.56
89194,89195,13655,16,97,25,Male,Student,270,462,4.23
89195,89196,24840,9,18,35,Male,Working Professional,230,819,3.77


In [75]:
train1=train.copy()
test1=test.copy()

In [76]:
for i in train:
    print(i,train[i].nunique())

row_id 89197
user_id 27734
category_id 47
video_id 175
age 58
gender 2
profession 3
followers 17
views 43
engagement_score 229


In [77]:
# train.row_id.nunique()

In [78]:
train=train.drop(columns='row_id')
test=test.drop(columns='row_id')

In [79]:
target = 'engagement_score'
categorical_cols=['gender', 'profession', 'age','user_id','followers','views']
continous_cols=['video_id']

In [80]:
# for e in ['gender', 'profession']:
#     le = LabelEncoder()
#     train[e]=le.fit_transform(train[e])
#     test[e]=le.transform(test[e])

In [81]:
# 

In [82]:
# train['video_id_eng_score']=train.groupby('category_id')['engagement_score'].transform('mean').fillna(0)
# i='video_id'
# mean_map=dict(zip(train.groupby([i])['engagement_score'].mean().reset_index()[i],train.groupby([i])['engagement_score'].mean().reset_index()['engagement_score']))
# test['video_id_eng_score']=test[i].map(mean_map)

In [83]:
def categorical_univariate_transformation(train,test,cat_features=None,Target=str):
    if cat_features!=None:
        train1=pd.DataFrame()
        test1=pd.DataFrame()
        train1=train.copy()
        test1=test.copy()
        try:
            cat_features=[i for i in cat_features if i not in Target]
        except:
            pass
        cats=cat_features
        a,b=np.meshgrid(cats,cats)
        m=pd.DataFrame({'a':a.flatten(),'b':b.flatten()})
        for i,j in zip(m['a'],m['b']):
            if(i!=j):
                train1[i+"_&_"+j]=train[i].astype(str)+"_"+train[j].astype(str)
                test1[i+"_&_"+j]=test[i].astype(str)+"_"+train[j].astype(str)
                #L.append(i+"_&_"+j)
        return train1,test1
    else:
        pass


In [84]:
train,test=categorical_univariate_transformation(train,test,cat_features=['gender','profession','age','followers'])

In [85]:
colstoadd=[i for i in train.columns if '&' in i]


In [86]:
for i in categorical_cols+colstoadd:
    mean_map=dict(zip(train.groupby([i])['engagement_score'].mean().reset_index()[i],train.groupby([i])['engagement_score'].mean().reset_index()['engagement_score']))
    train[i+'_'+'engagement_score'+'_mean']=train[i].map(mean_map)
    test[i+'_'+'engagement_score'+'_mean']=test[i].map(mean_map)
    max_map=dict(zip(train.groupby([i])['engagement_score'].max().reset_index()[i],train.groupby([i])['engagement_score'].max().reset_index()['engagement_score']))
    train[i+'_'+'engagement_score'+'_max']=train[i].map(max_map)
    test[i+'_'+'engagement_score'+'_max']=test[i].map(max_map)
    min_map=dict(zip(train.groupby([i])['engagement_score'].min().reset_index()[i],train.groupby([i])['engagement_score'].min().reset_index()['engagement_score']))
    train[i+'_'+'engagement_score'+'_min']=train[i].map(min_map)
    test[i+'_'+'engagement_score'+'_min']=test[i].map(min_map)
    std_map=dict(zip(train.groupby([i])['engagement_score'].std().reset_index()[i],train.groupby([i])['engagement_score'].std().reset_index()['engagement_score']))
    train[i+'_'+'engagement_score'+'_std']=train[i].map(std_map)
    test[i+'_'+'engagement_score'+'_std']=test[i].map(std_map)
    mean_map,max_map,min_map,std_map=0,0,0,0

In [87]:
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['gender', 'profession'],  # we can select which variables to encode
    drop_last=True)  # to return k-1, false to return k
train=train.drop(columns=colstoadd)
test=test.drop(columns=colstoadd)

In [88]:
train=train.fillna(0)
test=test.fillna(0)

In [89]:
train=ohe_enc.fit_transform(train.fillna('Missing').drop(columns=['engagement_score']))
test=ohe_enc.transform(test.fillna('Missing'))


In [90]:
train['engagement_score']=train1['engagement_score']

In [91]:
# data=train.drop(columns=[target])
# target=train[target]

In [92]:
# train=train.drop(columns=cols)
# test=test.drop(columns=cols)

In [93]:
# from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,StratifiedKFold
# lgb=LGBMRegressor(random_state=42)
# parameters={'n_estimators':np.arange(0,1000,100),
#            'learning_rate': [0.0145,0.01,0.001],
#             'colsample_bytree': [0.5,0.3,0.1,0.7],
#             'num_leaves':[5,10,15,20,25,30],
#             'min_child_samples':[20,30,40],
#             'min_child_weight': [0.001,0.0001,0.002,0.01]
#            }
# X=train.drop(columns=['engagement_score'])
# y=train['engagement_score']
# lgbrv=RandomizedSearchCV(lgb,parameters,cv=5,
#                               n_iter= 20,scoring='r2',random_state=2,verbose=1,n_jobs=-1)

In [94]:
# X_train, X_test, y_train, y_test = train_test_split(
#     train.drop(labels=['engagement_score'], axis=1),  # drop the target
#     train['engagement_score'],  # just the target
#     test_size=0.25,
#     random_state=0)
# lgbrv.fit(X_train,y_train)

In [95]:
# # preds=rf.fit(X_train,y_train).predict(test5)

# lgbrv.fit(X_train,y_train)
# xgb=lgbrv.best_estimator_

In [96]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [97]:
# cols

In [130]:
params= {'n_estimators':480,'reg_alpha': 9.208871583495077, 'reg_lambda': 0.005377048455752813, 'colsample_bytree': 0.6, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 10, 'num_leaves': 578, 'min_child_samples': 76, 'min_data_per_groups': 1,'random_state':42}

In [131]:
# train_data=train.select_dtypes([int,float])
# test_data=test.select_dtypes([int,float])
columns = [i for i in train.columns if i!='engagement_score']
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=10)
rmse=[]  # list contains rmse for each fold
n=0
feat_imp=pd.DataFrame()
count=1
for trn_idx, test_idx in kf.split(train[columns],train[target]):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train[target].iloc[trn_idx],train[target].iloc[test_idx]
    model = LGBMRegressor(**params)
    count+=1
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    feature_imp = pd.DataFrame({f'Value_{count}':model.feature_importances_,f'Feature':model.feature_name_})
    feat_imp=pd.concat([feat_imp,feature_imp],axis=1)
    preds+=model.predict(test[columns])/kf.n_splits
    print(r2_score(y_tr, model.predict(X_tr)))
    print(r2_score(y_val, model.predict(X_val)))
    n+=1

0.7415901291380688
0.7212956795783358


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7425271178978881
0.7189176305320932


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7436537120047749
0.705146231381042


...

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.74322744537367
0.7131262543320801


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7417964925091377
0.7233913535161388


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7428943030948006
0.7153599828055129


In [132]:
feat_imp1 = feat_imp.loc[:,~feat_imp.columns.duplicated()].copy()

In [133]:
feat_imp1['imp']=feat_imp1.mean(axis=1)

<ipython-input-133-aaf74bd9c10c>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  feat_imp1['imp']=feat_imp1.mean(axis=1)


In [134]:
feat_imp1=feat_imp1.sort_values(by=['imp'],ascending=False)

In [142]:
pd.set_option('display.max_rows',None)
cols=feat_imp1.iloc[0:56]['Feature'].values

In [143]:
columns = cols
preds = np.zeros(test.shape[0])
kf = KFold(n_splits=10)
rmse=[]  # list contains rmse for each fold
n=0
feat_imp=pd.DataFrame()
count=1
for trn_idx, test_idx in kf.split(train[columns],train[target]):
    X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
    y_tr,y_val=train[target].iloc[trn_idx],train[target].iloc[test_idx]
    model = LGBMRegressor(**params)
    count+=1
    model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
    feature_imp = pd.DataFrame({f'Value_{count}':model.feature_importances_,f'Feature':model.feature_name_})
    feat_imp=pd.concat([feat_imp,feature_imp],axis=1)
    preds+=model.predict(test[columns])/kf.n_splits
    print(r2_score(y_tr, model.predict(X_tr)))
    print(r2_score(y_val, model.predict(X_val)))
    n+=1

C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7418230060211756
0.7214260128638422
[LightGBM] [Warning] Unknown parameter: min_data_per_groups


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


0.7424224415539292
0.7194593090293357


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7430954304795322
0.7129637969921804


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7418992568513384
0.7235850657692513


C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\EG569LZ\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: min_data_per_groups
0.7430171164261146
0.7157570936604811


In [ ]:
# columns = [i for i in train.columns if i!='engagement_score']
# preds = np.zeros(test.shape[0])
# kf = KFold(n_splits=5,random_state=48,shuffle=True)
# rmse=[]  # list contains rmse for each fold
# n=0
# for trn_idx, test_idx in kf.split(train[columns],train[target]):
#     X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
#     y_tr,y_val=train[target].iloc[trn_idx],train[target].iloc[test_idx]
#     model = LGBMRegressor(**params)
#     model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
#     preds+=model.predict(test[columns])/kf.n_splits
#     print(r2_score(y_tr, model.predict(X_tr)))
#     print(r2_score(y_val, model.predict(X_val)))
#     n+=1

In [144]:
preds

array([4.20534504, 3.95687806, 2.38998311, ..., 2.97093188, 3.80135401,
       2.79179293])

In [146]:
submission=pd.read_csv('sample_submission_JPlpRcN.csv')
submission['row_id']=test1['row_id']
submission['engagement_score']=preds

In [147]:
submission.to_csv('submission3.csv',index=False)

In [ ]:
# from lightgbm import LGBMRegressor
# from sklearn.model_selection import GridSearchCV,RandomizedSearchCV, KFold,cross_val_score,StratifiedKFold,cross_validate
# from sklearn.preprocessing import MaxAbsScaler,StandardScaler
# lgb=LGBMRegressor(random_state=10,max_depth=10)
# from sklearn.metrics import mean_squared_error,r2_score
# def run_reg_kfold(train,test,model_name=None,target=list(),pred='engagement_score'):
#     pred=pred
#     predictions = np.zeros(test.shape[0])
#     oobs = np.zeros(train.shape[0])
#     y = train[pred].values
#     #shuffle=True, random_state=123
#     repeat=3
#     #skf = KFold(n_splits=10,shuffle=True, random_state=123)
#     skf=KFold(n_splits=10)
#     model = model_name
#     X=train.drop(columns=pred).copy()
#     y=train[pred].copy()
#     print('Regression started')
#     X_test=test.drop(columns=target)
#     transformer = StandardScaler().fit(X)
#     X = transformer.transform(X)
#     X_test = transformer.transform(X_test)
#     for i, (train_id, valid_id) in enumerate(skf.split(X,y)):
#         print(f"fold {i}")
#         X_train, y_train = X[train_id], y[train_id]
#         X_valid, y_valid = X[valid_id], y[valid_id]
#         model.fit(X_train, y_train, eval_set =[(X_valid, y_valid)],  early_stopping_rounds=200, verbose=1000, eval_metric='r2')
#         oobs[valid_id] = model.predict(X_valid)
#         train_preds=model.predict(X_train)
#         valid_preds=model.predict(X_valid)
#         print(r2_score(y_train,train_preds))
#         print(r2_score(y_valid,valid_preds))
#         predictions += model.predict(X_test)
# #     roc_auc = roc_auc_score(y,oobs)
#     finals = predictions/(10)
#     lgb_predictions = finals
#     return lgb_predictions

In [ ]:
run_reg_kfold(train_data,test_data,model_name=lgb,target=list(),pred='engagement_score')

In [ ]:
pd.to_datetime('11-13-2010', format='%d-%m-%Y')

In [ ]:
def fast(x=[]):
    x.append(1)
    return x
print(fast())
print(fast())